In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
df = pd.read_excel('2013-2019_WR_Data.xlsx')

In [ ]:
df = df.drop(['Player'], axis=1)
df = df.iloc[np.random.permutation(len(df))]

In [ ]:
train_df = df[:160]
test_df = df[160:]

In [ ]:
train_df['5_x'] = train_df['5_x'].replace(',','',regex=True)
train_df['4_y'] = train_df['4_y'].replace(',','',regex=True)
test_df['5_x'] = test_df['5_x'].replace(',','',regex=True)
test_df['4_y'] = test_df['4_y'].replace(',','',regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
train_df = train_df.astype(float)
test_df = test_df.astype(float)

In [ ]:
train_average_points = train_df.pop('Following_Year_PPG')

test_average_points = test_df.pop('Following_Year_PPG')

In [ ]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((train_df.values, train_average_points.values))
train_dataset = dataset.shuffle(len(df)).batch(1)

dataset = tf.data.Dataset.from_tensor_slices((test_df.values, test_average_points.values))
test_dataset = dataset.shuffle(len(df)).batch(1)

In [ ]:
from tensorflow import keras
def get_compiled_model():
  model = tf.keras.Sequential([
    keras.layers.Flatten(input_shape=(15,)),
    keras.layers.Dense(20, activation='relu'),
	  keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.GaussianNoise(0.1),
    keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=50)

Epoch 1/50
160/160 [==============================] - 1s 1ms/step - loss: 77.1981 - mae: 6.6861
Epoch 2/50
160/160 [==============================] - 0s 1ms/step - loss: 41.1553 - mae: 5.0907
Epoch 3/50
160/160 [==============================] - 0s 1ms/step - loss: 44.1014 - mae: 5.3143
Epoch 4/50
160/160 [==============================] - 0s 1ms/step - loss: 30.3948 - mae: 4.5289
Epoch 5/50
160/160 [==============================] - 0s 2ms/step - loss: 27.9741 - mae: 4.3397
Epoch 6/50
160/160 [==============================] - 0s 1ms/step - loss: 25.5634 - mae: 4.0801
Epoch 7/50
160/160 [==============================] - 0s 1ms/step - loss: 23.2500 - mae: 3.7927
Epoch 8/50
160/160 [==============================] - 0s 1ms/step - loss: 23.9675 - mae: 3.9125
Epoch 9/50
160/160 [==============================] - 0s 1ms/step - loss: 30.2618 - mae: 4.2281
Epoch 10/50
160/160 [==============================] - 0s 1ms/step - loss: 22.9271 - mae: 3.8272
Epoch 11/50
160/160 [==================

In [ ]:
test_loss, test_acc = model.evaluate(test_df, test_average_points)
print('Test accuracy:', test_acc)

2/2 [==============================] - 0s 7ms/step - loss: 30.1045 - mae: 4.4196
Test accuracy: 4.419551849365234


In [ ]:
while True:
  model = get_compiled_model()
  model.fit(train_dataset, epochs=50)
  test_loss, test_acc = model.evaluate(test_df, test_average_points)
  print('Test accuracy:', test_acc)
  if test_acc < 2:
    break

In [ ]:
model.save('Newest_Accurate_WR_Model.h5')